In [17]:
sequences = []

with open("sample/test_brenda/sampled_100799.txt", "r") as f:
    for line in f.readlines():
        sequences.append(line.rstrip('\n'))

In [31]:
from utils.constants import ID_TO_AMINO_ACID, NON_STANDARD_AMINO_ACIDS

def convert_to_string(sequence):
    return "".join([ID_TO_AMINO_ACID[i] for i in sequence])

In [24]:
seq_mod = []

for seq in sequences:
    #temp_seq = convert_to_string(seq)
    temp_seq = seq.replace("0", "")
    seq_mod.append(temp_seq)

In [25]:
seq_mod

['...']

In [44]:
import pandas as pd

def load_dataset_ecfp(max_length, max_n_examples, data_dir=''):
    '''Adapted from https://github.com/igul222/improved_wgan_training/blob/master/language_helpers.py'''
    print ("loading dataset...")

    temp_df = pd.read_csv(data_dir)
    temp_df = temp_df.dropna(subset=["ECFP"])
    
    if len(temp_df) > max_n_examples:
        temp_df = temp_df[:max_n_examples]
    
    temp_seq = list(temp_df["Sequence"])
    temp_ectp = list(temp_df["ECFP"])
    temp_logkm = list(temp_df["log10_KM"])

    seqs = []

    for line, ecfp_line, logkm in zip(temp_seq, temp_ectp, temp_logkm):
        line = line.split(" ")[0].strip()
        if ~any(ext in line for ext in NON_STANDARD_AMINO_ACIDS):
            line = tuple(line)

            if len(line) > max_length:
                line = line[:max_length]
            

            #line_out = line + ( ("0",)*(max_length-len(line)) )
            seqs.append("".join(line))
    
    return seqs

In [45]:
seq_len = 512
data_dir="./data/new_brenda_df_data.csv"

def load_data(datadir):
    max_examples = 1e6
    data = load_dataset_ecfp(
        max_length=seq_len,
        max_n_examples=max_examples,
        data_dir=datadir,
    )

    return data

main_data = load_data(datadir=data_dir)

loading dataset...


In [46]:
main_data[0]

'MQNFTFRNPTKLIFGRGQIEQLKEEVPKYGKKVLLVYGGGSIKRNGLYDEVMSLLTDIGAEVVELPGVEPNPRLSTVKKGVDICRREGIEFLLAVGGGSVIDCTKAIAAGAKFDGDPWEFITKKATVTEALPFGTVLTLAATGSEMNAGSVITNWETKEKYGWGSPVTFPQFSILDPTYTMTVPKDHTVYGIVDMMSHVFEQYFHHTPNTPLQDRMCEAVLKTVIEAAPKLVDDLENYELRETIMYSGTIALNGFLQMGVRGDWATHDIEHAVSAVYDIPHAGGLAILFPNWMKHVLDENVSRFAQLAVRVFDVDPTGKTERDVALEGIERLRAFWSSLGAPSRLADYGIGEENLELMADKAMAFGEFGRFKTLNRDDVLAILRASL'

In [53]:
from rdkit.Chem import AllChem
from rdkit import DataStructs
import numpy as np

def remap(x, x_min, x_max):
        return (x - x_min) / (x_max - x_min)

def __compute_diversity(seq, fps):
    ref_fps = AllChem.GetMorganFingerprintAsBitVect(seq, 4, nBits=2048)
    dist = DataStructs.BulkTanimotoSimilarity(ref_fps, fps, returnDistance=True)
    score = np.mean(dist)
    return score

def diversity_scores(seqs, data):
    rand_seqs = np.random.choice(data, 100)
    print(rand_seqs[0])
    fps = [AllChem.GetMorganFingerprintAsBitVect(seq, 4, nBits=2048) for seq in rand_seqs]

    scores = np.array(
        list(map(lambda x: __compute_diversity(x, fps) if x is not None else 0, seqs)))
    scores = np.clip(remap(scores, 0.9, 0.945), 0.0, 1.0)

    return scores

In [ ]:
div_scores = diversity_scores(seq_mod, main_data)

## Find Global and Local alignment Simialrity
https://towardsdatascience.com/pairwise-sequence-alignment-using-biopython-d1a9d0ba861f

In [61]:
from Bio import pairwise2
from Bio.pairwise2 import format_alignment

X = "MQKIKRGEYLRQLMTLKTLINQLVKEFAAEAEKCFVKTLAGAELADEQDALEDILAEFSGEYCSLESWLLGEEQPPRRNTLGASRIVVAWADSKRPIKTYKSAVNGLCGTLTHDKTFDPRSLNAANQDEPLMRWAISNGKLGEAAAWLQPWSLDARVLACITEAPCLAHTLAAVLRRGGWADPVFVAKLARDTAMKETIYANFEAVPMAQRKRAGVAGMWLANYDSAAL"
Y = "MISASLQQRKTRTRRSMLFVPGANAAMVSNSFIYQADALMFDLEDSVALREKDTARRMVYHALQHPLYRDIETIVRVNALDSEWGVNDLEAVVRGGADVVRLPKTDTAQDVLDIEKEILRIEKACGREPGSTGLLAAIESPLGITRAVEIAHASERLIGIALGAEDYVRNLRTERSPEGTELLFARCSILQAARSAGIQAFDTVYSDANNEAGFLQEAAHIKQLGFDGKSLINPRQIDLLHNLYAPTQKEVDHARRVVEAAEAAAREGLGVVSLNGKMVDGPVIDRARLVLSRAELSGIREE"

alignments = pairwise2.align.globalxx(X, Y)

for a in alignments:
    print(format_alignment(*a))

MQKI-------K-R---------GE-----------YLRQ---LMT--L--------K-T--------LINQ--LVKEFAA----E------A---EKCF--VKT--L-A---G-AE----LA---DE--QDA-LEDI----LA--EFS---G-EYC--S--L----ESWL-LG-----E-----EQPPRRNTL-GASRIVVAW--A-DSKRPIKTYKSAV-NGLC-------GT-LTHDKT-FDP-R-S-LN-AANQDEPLMRW-A-I-------S--N---GK-LG-EAAAWL---QPWSL--DARV---LACITEA-PC----LA-HT-LA-A-----VL---RRGGWADPVFVAKL-ARDTAMKETIYANFEAVPMAQRKRA-GVA-GMW---LANY----DS------A------A-L------
|  |       | |         |            |  |   ||   |        | |        |  |  |          |      |   |     |    | |   | |     |    |   ||  | ||    |   |     | |    |  |    ||   ||     |     |    |  | |   |  |   | |       |   | | |        || |      |   | | |  ||       |  | |       |  |   |  |  | ||     |   |  |      |  |    |     |  |  |  |     |    ||      | |    |   |  |   |   |   |   |  |   |     | |     |       |      | |      
M--ISASLQQRKTRTRRSMLFVPG-ANAAMVSNSFIY--QADALM-FDLEDSVALREKDTARRMVYHAL--QHPL------YRDIETIVRVNALDSE---WGV--NDLEAVVRGGA-DVVRL-PKTD-TAQD-VL-DIEK

### Local alignments

In [62]:
X = "MQKIKRGEYLRQLMTLKTLINQLVKEFAAEAEKCFVKTLAGAELADEQDALEDILAEFSGEYCSLESWLLGEEQPPRRNTLGASRIVVAWADSKRPIKTYKSAVNGLCGTLTHDKTFDPRSLNAANQDEPLMRWAISNGKLGEAAAWLQPWSLDARVLACITEAPCLAHTLAAVLRRGGWADPVFVAKLARDTAMKETIYANFEAVPMAQRKRAGVAGMWLANYDSAAL"
Y = "MISASLQQRKTRTRRSMLFVPGANAAMVSNSFIYQADALMFDLEDSVALREKDTARRMVYHALQHPLYRDIETIVRVNALDSEWGVNDLEAVVRGGADVVRLPKTDTAQDVLDIEKEILRIEKACGREPGSTGLLAAIESPLGITRAVEIAHASERLIGIALGAEDYVRNLRTERSPEGTELLFARCSILQAARSAGIQAFDTVYSDANNEAGFLQEAAHIKQLGFDGKSLINPRQIDLLHNLYAPTQKEVDHARRVVEAAEAAAREGLGVVSLNGKMVDGPVIDRARLVLSRAELSGIREE"

alignments = pairwise2.align.localxx(X, Y)

for a in alignments:
    print(format_alignment(*a))

1 MQKI-------K-R---------GE-----------YLRQ---LMT--L--------K-T--------LINQ--LVKEFAA----E------A---EKCF--VKT--L-A---G-AE----LA---DE--QDA-LEDI----LA--EFS---G-EYC--S--L----ESWL-LG-----E-----EQPPRRNTL-GASRIVVAW--A-DSKRPIKTYKSAV-NGLC-------GT-LTHDKT-FDP-R-S-LN-AANQDEPLMRW-A-I-------S--N---GK-LG-EAAAWL---QPWSL--DARV---LACITEA-PC----LA-HT-LA-A-----VL---RRGGWADPVFVAKL-ARDTAMKETIYANFEAVPMAQRKRA-GVA-GMW---LANY----DS------A------A-L
  |  |       | |         |            |  |   ||   |        | |        |  |  |          |      |   |     |    | |   | |     |    |   ||  | ||    |   |     | |    |  |    ||   ||     |     |    |  | |   |  |   | |       |   | | |        || |      |   | | |  ||       |  | |       |  |   |  |  | ||     |   |  |      |  |    |     |  |  |  |     |    ||      | |    |   |  |   |   |   |   |  |   |     | |     |       |      | |
1 M--ISASLQQRKTRTRRSMLFVPG-ANAAMVSNSFIY--QADALM-FDLEDSVALREKDTARRMVYHAL--QHPL------YRDIETIVRVNALDSE---WGV--NDLEAVVRGGA-DVVRL-PKTD-TAQD-VL-DIEKEIL-RI

In [70]:
X = "MQKIKRGEYLRQLMTLKTLINQLVKEFAAEAEKCFVKTLAGAELADEQDALEDILAEFSGEYCSLESWLLGEEQPPRRNTLGASRIVVAWADSKRPIKTYKSAVNGLCGTLTHDKTFDPRSLNAANQDEPLMRWAISNGKLGEAAAWLQPWSLDARVLACITEAPCLAHTLAAVLRRGGWADPVFVAKLARDTAMKETIYANFEAVPMAQRKRAGVAGMWLANYDSAAL"
Y = "MISASLQQRKTRTRRSMLFVPGANAAMVSNSFIYQADALMFDLEDSVALREKDTARRMVYHALQHPLYRDIETIVRVNALDSEWGVNDLEAVVRGGADVVRLPKTDTAQDVLDIEKEILRIEKACGREPGSTGLLAAIESPLGITRAVEIAHASERLIGIALGAEDYVRNLRTERSPEGTELLFARCSILQAARSAGIQAFDTVYSDANNEAGFLQEAAHIKQLGFDGKSLINPRQIDLLHNLYAPTQKEVDHARRVVEAAEAAAREGLGVVSLNGKMVDGPVIDRARLVLSRAELSGIREE"

alignments = pairwise2.align.globalms(X, Y, 2, -1, -0.5, -0.1)

#### Score, source: https://www.biostars.org/p/208540/
print("Score: ", alignments[0][2])
#for a in alignments:
#    print(format_alignment(*a))

seq_length = min(len(X), len(Y))
matches = alignments[0][2]

percent_match = (matches / seq_length) * 100
print("Percent Match: ", percent_match)

Score:  126.10000000000046
Percent Match:  55.06550218340631


### Calculating Needle and Water Commandline

In [72]:
from Bio.Emboss.Applications import NeedleCommandline
from Bio.Emboss.Applications import WaterCommandline
import re

In [101]:
def needle_align_code(query_seq, target_seq):
    needle_cline = NeedleCommandline(asequence="asis:" + query_seq,
                                     bsequence="asis:" + target_seq,
                                     aformat="simple",
                                     gapopen=10,
                                     gapextend=0.5,
                                     outfile='stdout'
                                     )
    out_data, err = needle_cline()
    out_split = out_data.split("\n")
    p = re.compile("\((.*)\)")
    return p.search(out_split[25]).group(1).replace("%", "")

# needle_align_code(X, Y)

In [102]:
def water_align(query_seq, target_seq):
    needle_cline = WaterCommandline(asequence="asis:" + query_seq,
                                    bsequence="asis:" + target_seq,
                                    aformat="simple",
                                    gapopen=10,
                                    gapextend=0.5,
                                    outfile='stdout'
                                    )
    out_data, err = needle_cline()
    out_split = out_data.split("\n")
    p = re.compile("\((.*)\)")
    return p.search(out_split[25]).group(1).replace("%", "")

# water_align(Y, X)

In [103]:
def hamming_distance(str1, str2):
    #assert len(str1) == len(str2)
    return sum(chr1 != chr2 for chr1, chr2 in zip(str1, str2))

# hamming_distance(X, Y)

### Calculating global alignment similarity score of every sequences

In [ ]:
# alignments = pairwise2.align.globalms(X, Y, 2, -1, -0.5, -0.1)

# seq_length = min(len(X), len(Y))
# matches = alignments[0][2]

# percent_match = (matches / seq_length) * 100
# print("Percent Match: ", percent_match)

In [83]:
def __compute_global_aligment_diversity(seq, rand_seqs):
    g_aligns = []
    percent_matches = []
    for r_seq in rand_seqs:
        alignments = pairwise2.align.globalms(seq, r_seq, 2, -1, -0.5, -0.1)
        g_aligns.append(alignments[0][2])

        seq_length = min(len(seq), len(r_seq))
        matches = alignments[0][2]
        percent_match = (matches / seq_length) * 100

        percent_matches.append(percent_match)

    score = np.mean(g_aligns)
    percent = np.mean(percent_matches)
    return score, percent

In [97]:
def global_alignment_similarity_scores(seqs, data):
    rand_seqs = np.random.choice(data, 100)
    #print(rand_seqs[0])

    scores_list = np.array(
        list(map(lambda x: __compute_global_aligment_diversity(x, rand_seqs) if x is not None else 0, seqs)))

    return scores_list

In [98]:
temp_out = global_alignment_similarity_scores(seq_mod[:32], main_data)

In [99]:
temp_out

array([[178.197     ,  53.58435888],
       [149.495     ,  51.09964782],
       [186.643     ,  54.98965778],
       [129.207     ,  52.55195437],
       [183.473     ,  54.4227927 ],
       [150.691     ,  53.19812534],
       [164.791     ,  51.86284395],
       [186.907     ,  55.28530356],
       [144.639     ,  50.161163  ],
       [175.773     ,  52.86026266],
       [140.629     ,  51.78298116],
       [ 97.369     ,  63.04975707],
       [182.295     ,  54.25533256],
       [126.117     ,  55.24794292],
       [143.565     ,  50.67897271],
       [188.647     ,  55.59242965],
       [123.757     ,  52.8696272 ],
       [153.815     ,  50.6582399 ],
       [136.177     ,  51.94322603],
       [137.107     ,  51.73907028],
       [172.703     ,  52.87104073],
       [146.693     ,  51.24003421],
       [175.123     ,  53.33844773],
       [174.849     ,  53.00591657],
       [159.423     ,  52.30974622],
       [158.121     ,  51.00065718],
       [119.133     ,  51.45055315],
 